# NER 

In [1]:
!pip install datasets

In [2]:
from datasets import get_dataset_config_names

xtreme_subsets = get_dataset_config_names("google/xtreme")
print(f"XTREME has {len(xtreme_subsets)} dataset subsets")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

XTREME has 183 dataset subsets


In [3]:
panx_subsets = [s for s in xtreme_subsets if s.startswith("PAN")]
panx_subsets[:3]

['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg']

### Each of the subset has 2 letter suffix that apears to be language code 
### To load german corpus we pass "de" code to name argument of load dataset()

In [4]:
from datasets import load_dataset
panx = load_dataset("google/xtreme", "PAN-X.de")


Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)


Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)


Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)


Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)



PAN-X.de/train-00000-of-00001.parquet:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

PAN-X.de/validation-00000-of-00001.parqu(…):   0%|          | 0.00/590k [00:00<?, ?B/s]

PAN-X.de/test-00000-of-00001.parquet:   0%|          | 0.00/588k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

As our problem statement is NER in switzerland , we will sample  German ,french, Italian, English according to thier spoken proportions  
So there will be an imbalance, This imbalanced dataset will simulate a comman situation when working on multilingual applications ,and we can see how we can build a model that works on all languages

### Lets create a Python defaultdict that stores the language code as the key and a PAN-X corpus of type DatasetDict as the value

In [5]:
from collections import defaultdict
from datasets import DatasetDict

langs = ["de", "fr", "it", "en"]
fracs = [0.629, 0.229, 0.084, 0.059]

# Returns a DatasetDict if a key doesn't exist
panx_ch = defaultdict(DatasetDict)

for lang, frac in zip(langs, fracs):
    # load monolingual dataset
    ds  = load_dataset("google/xtreme", name =f"PAN-X.{lang}")
    # Shuffle and downsample each split according to fraction
    for split in ds:
        panx_ch[lang][split] = (
            ds[split]
            .shuffle(seed=0)
            .select(range(int(frac * ds[split].num_rows))))
            

PAN-X.fr/train-00000-of-00001.parquet:   0%|          | 0.00/837k [00:00<?, ?B/s]

PAN-X.fr/validation-00000-of-00001.parqu(…):   0%|          | 0.00/419k [00:00<?, ?B/s]

PAN-X.fr/test-00000-of-00001.parquet:   0%|          | 0.00/423k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

...

PAN-X.en/validation-00000-of-00001.parqu(…):   0%|          | 0.00/472k [00:00<?, ?B/s]

PAN-X.en/test-00000-of-00001.parquet:   0%|          | 0.00/472k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [6]:
import pandas as pd
pd.DataFrame(
    {lang: panx_ch[lang]["train"].num_rows for lang in langs},
    index=["# Train examples"]
)

,de,fr,it,en
# Train examples,12580,4580,1680,1180


We have more German samples . We use it aas starting point to perform  *Zero-shot cross lingual transfer* to other languages

Lets inspect one of the examples in German corpus


In [7]:
element = panx_ch["de"]["train"][0]

for key,value in element.items():
    print(f"{key}: {value}")

tokens: ['2.000', 'Einwohnern', 'an', 'der', 'Danziger', 'Bucht', 'in', 'der', 'polnischen', 'Woiwodschaft', 'Pommern', '.']
ner_tags: [0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0]
langs: ['de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de']


This is cryptic (ner_tags) so lets create a new Column with LOC, PER, ORG tags .

Dataset object has a features attribute that specifies underlying data types assosiated with each column

In [8]:
for key,value in panx_ch["de"]["train"].features.items():
    print(f"{key}: {value}")

tokens: List(Value('string'))
ner_tags: List(ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']))
langs: List(Value('string'))


In [9]:
tags = panx_ch["de"]["train"].features["ner_tags"].feature
print(tags)

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'])


We can use a ClassLabel.int2str() method to create a new column i our training set with class names for each tag

We will use map() to return a dict with the key corresponding to new column name and values as list of class names 

In [10]:
def create_tag_names(batch):
    return {"ner_tags_str": [tags.int2str(idx)for idx in batch["ner_tags"]]}

panx_de = panx_ch["de"].map(create_tag_names)

Parameter 'function'=<function create_tag_names at 0x79f89f3fcfe0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/12580 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

In [11]:
de_example = panx_de["train"][0]
pd.DataFrame([de_example["tokens"], de_example["ner_tags_str"]],["Tokens", "Tags"])

,0,1,2,3,4,5,6,7,8,9,10,11
Tokens,2.000,Einwohnern,an,der,Danziger,Bucht,in,der,polnischen,Woiwodschaft,Pommern,.
Tags,O,O,O,O,B-LOC,I-LOC,O,O,B-LOC,B-LOC,I-LOC,O


We have to check for any unusual imbalance in tags.

 lets calculate fequiencies of each entity across each split 

In [12]:
from collections import Counter

split2freqs = defaultdict(Counter)
for split,dataset  in panx_de.items():
    for row in dataset["ner_tags_str"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] +=1

pd.DataFrame.from_dict(split2freqs, orient= "index") 

,LOC,ORG,PER
train,6186,5366,5810
validation,3172,2683,2893
test,3180,2573,3071


NOW WE CAN USE MULTILINGUAL TRANSFORMER 

XLM-R is a great choice for multilingual NLU tasks 

XLM-R uses  a SentencePiece tokenizer trainined on raw text of 100 languages 



## Comparison b/w SentencePiece and WordPiece

In [13]:
from transformers import AutoTokenizer

bert_model_name = "bert-base-cased"
xlmr_model_name = "xlm-roberta-base"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [14]:
text = "Jack sparrow loves New York!"

bert_tokens = bert_tokenizer(text).tokens()
xlmr_tokens = xlmr_tokenizer(text).tokens()
print("BERT tokens:", bert_tokens)
print("XLM-R tokens:", xlmr_tokens)


BERT tokens: ['[CLS]', 'Jack', 'spa', '##rrow', 'loves', 'New', 'York', '!', '[SEP]']
XLM-R tokens: ['<s>', '▁Jack', '▁spar', 'row', '▁love', 's', '▁New', '▁York', '!', '</s>']


## Creating a custom model for Token Classification



In [16]:
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel, RobertaModel

class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    config_class = XLMRobertaConfig

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        # Load model body
        self.roberta = RobertaModel(config, add_pooling_layer=False) 
        # Token classification head
        self.dropout = nn.Dropout(config.hidden_dropout_prob)   
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        # Initialize weights
        self.init_weights()
    
    def forward(self,input_ids=None, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        # Use model body to get hidden states
        outputs = self.roberta(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, **kwargs)
        # Apply dropout to hidden states
        sequence_output = self.dropout(outputs[0])
        # Get logits from classifier
        logits = self.classifier(sequence_output)
        loss = None
        if labels is not None:
            # Define loss function
            loss_fct = nn.CrossEntropyLoss()
            # Compute loss, ignoring padded tokens (label = -100)
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions)
    

With the super() method we call the initialization fn of the RobertaPreTrainedModel class .

During the forward pas, data first fed through the model body.There are no. of input variables only one we need in this stage is input ids and attention mask.

hidden state which is the part of model body output is then fed through the dropout and classification layer.

If We also  pass labels we can calculate loss too..

If there is attention mask we need to do a extra work to calculate loss only  on unmasked tokens .

Finally we wrap the output in  TokenClassifierOutput.